# 1. 토큰화

1. stemming: 일일이 어미를 찾아 제외하는 규칙 기반 방식 어간 추출
2. lemmatization: 알려진 단어 형태 사전을(명시적이고 사람이 구축한 시스템) 사용하고 문장에서 단어의 역할을 고려
     - 정규화: 두 처리 방식, 표제어 추출과 어간 추출은 단어의 정규화의 한 형태로 볼 수 있다.

In [1]:
import spacy
import nltk

In [9]:
# spacy의 영어 모델을 로드합니다.
en_nlp = spacy.load('en')

# nltk PorterStemmer 객체 생성한다.
stemmer = nltk.stem.PorterStemmer()

# spacy 표제어 추출과 nltk 어간 추출을 비교하는 함수
def compare_normalization(doc):
    doc_spacy = en_nlp(doc)
    
    print("표제어: ")
    print([token.lemma_ for token in doc_spacy])
    
    print("어간: ")
    print([stemmer.stem(token.norm_.lower()) for token in doc_spacy])

In [10]:
compare_normalization(u"Our meeting today was worse than yesterday," 
                       "i'm scared of meeting the clients tomrrow. ")

표제어: 
['-PRON-', 'meeting', 'today', 'be', 'bad', 'than', 'yesterday', ',', "i'm", 'scar', 'of', 'meet', 'the', 'client', 'tomrrow', '.']
어간: 
['our', 'meet', 'today', 'wa', 'wors', 'than', 'yesterday', ',', "i'm", 'scare', 'of', 'meet', 'the', 'client', 'tomrrow', '.']


- 표제어 추출은 어간 추출보다 훨씬 복잡한 처리를 거친다. 하지만 머신러닝을 위해 토큰 정규화를 할 때는 어간 추출보다 좋은 결과를 낸다.